In [1]:
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

--2024-07-01 22:58:55--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py.4’

minsearch.py.4      100%[===================>]   3.74K  --.-KB/s    in 0s      

2024-07-01 22:58:55 (14.5 MB/s) - ‘minsearch.py.4’ saved [3832/3832]



In [2]:
import minsearch

In [3]:
import json

In [4]:
with open('documents.json', 'rt') as f_in:
    raw_doc = json.load(f_in)

In [5]:
documents = []

for course_dict in raw_doc:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [6]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [7]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

In [8]:
index.fit(documents)

In [9]:
import os
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage

api_key = os.environ["MISTRAL_API_KEY"]
client = MistralClient(api_key=api_key)

In [10]:
def search_engine(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
            query=query,
            filter_dict={'course': 'data-engineering-zoomcamp'},
            boost_dict=boost,
            num_results=5
        )
    return results

In [11]:
def construct_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()
    context=""
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
        
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [12]:
def ask_llm(prompt):
    model = "mistral-large-latest"
    
    chat_response = client.chat(
        model=model,
        messages=[ChatMessage(role="user", content=prompt)]
    )
    return chat_response.choices[0].message.content

In [13]:
def rag(query):
    search = search_engine(query)
    prompt = construct_prompt(query, search)
    response = ask_llm(prompt)
    return response
    

In [14]:
query = 'the course has already started, can I still enroll?'
rag(query)

"Yes, you can still enroll in the course even if it has already started. You're eligible to submit homeworks, but please be aware that there will be deadlines for turning in the final projects. It's recommended not to leave everything for the last minute."

In [26]:
from elasticsearch import Elasticsearch

In [27]:
es_client = Elasticsearch('http://localhost:9200') 

In [28]:
es_client.info()

ObjectApiResponse({'name': '4a439f3687f4', 'cluster_name': 'docker-cluster', 'cluster_uuid': '0qIC96-8ROqTakKgn3JB1w', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [29]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [30]:
from tqdm.auto import tqdm

In [31]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

In [32]:
query = 'I just disovered the course. Can I still join it?'

In [33]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [36]:
def rag(query):
    search = elastic_search(query)
    prompt = construct_prompt(query, search)
    response = ask_llm(prompt)
    return response

In [37]:
rag(query)

"Yes, you can still join the course even if it has already started. You're eligible to submit homeworks, but please be aware that there will be deadlines for turning in the final projects. So, it's recommended not to leave everything for the last minute. You don't need to register to start learning and submitting homework. Registration is just to gauge interest before the start date."